In [1]:
from dotenv import load_dotenv
import os

# 1. 현재 경로의 .env 파일 로드
load_dotenv()

# 2. 환경 변수 읽기
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

In [2]:
from openai import OpenAI

client = OpenAI(api_key=OPENAI_API_KEY)
client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {
            "role": "user", 
            "content": "Who won the world series in 2020?"
        }
    ]
)

ChatCompletion(id='chatcmpl-CTkOQb76BSTwFzt9pY9qiFLsqQp3r', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content="The Los Angeles Dodgers won the World Series in 2020. They defeated the Tampa Bay Rays in six games to secure the championship. This victory marked the Dodgers' first World Series title since 1988.", refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=None))], created=1761205882, model='gpt-4o-mini-2024-07-18', object='chat.completion', service_tier='default', system_fingerprint='fp_560af6e559', usage=CompletionUsage(completion_tokens=42, prompt_tokens=17, total_tokens=59, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0)))

In [5]:
from langchain_community.document_loaders import Docx2txtLoader

In [24]:
# 문서 로드
loader = Docx2txtLoader(r'../data/langchainThon/cv/경력기술서_천승우.docx')
pages = loader.load()

In [50]:
from langchain_text_splitters import CharacterTextSplitter

text_splitter = CharacterTextSplitter(
    separator=' ',
    chunk_size=500,
    chunk_overlap=50,
    length_function=len,
                 
)

In [51]:
texts = text_splitter.split_documents(pages)
print(texts[0])

page_content='경력기술서

[개인 정보]

이메일: fourleaves8@gmail.com

연락처: 010-4788-7980



[전문 분야]

• 데이터 사이언스 		 • AI 엔지니어링	

• 데이터 분석 		 • 프롬프트 엔지니어링

• 생물분자 및 화학공학 연구



[요약]

저는 생명·의학 데이터를 처리하고 분석 플랫폼을 구축한 경험 및 복잡한 데이터를 효율적으로 분석해 의사결정을 지원하는 역량을 갖추고 있습니다. Python 기반의 데이터 전처리, 알고리즘 고도화, 분류 및 예측 모델 개발을 통해 항암제 신규 적응증 탐색 플랫폼을 성공적으로 구현한 바 있습니다. 또한 데이터 수집·정규화·시각화 전 과정을 주도하며, 스타트업의 코스닥 상장 및 실제 제약사 프로젝트에 기여한 실무 경험을 쌓았습니다. 



[보유 역량 및 기술]

프로그래밍 언어

Python, R, SQL, Java

도구 및 프레임워크

pandas, NumPy, Matplotlib, seaborn, Plotly,' metadata={'source': '../data/langchainThon/cv/경력기술서_천승우.docx'}


In [52]:
print(texts[1])

page_content='프레임워크

pandas, NumPy, Matplotlib, seaborn, Plotly, scikit-learn, PyTorch, TensorFlow, RAG, LangChain, MySQL, Oracle

운영체제

Linux(Ubuntu), Windows, macOS

Palantir Foundry

Pipeline Builder, Contour, Ontology(Action), Workshop, AIP, Dataset

기타

Vim / Vi, Git / GitHub, Bioinformatics Tools (Gene Set Enrichment Analysis, Bismark, Metilene, lifelines, survival, survminer, lifelines)



[주요 경력 및 프로젝트]

(주)온코크로스 · AI 연구소 · 주임연구원–전임연구원 · 2021.01–2024.12 (4년간 근무)



<프로젝트1 : 항암제 신규 적응증 탐색 플랫폼 개발' metadata={'source': '../data/langchainThon/cv/경력기술서_천승우.docx'}
